Your first task is to install Biogeme. Instructions are given on the package webpage: https://biogeme.epfl.ch/install.html

In [2]:
# Load packages
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.messaging as msg
from biogeme.expressions import Beta, Variable, Derive

# Change me after the first run to save some time
FIRST_RUN = True

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if FIRST_RUN:
    # Load in the trips csv file
    # As written, this will search back one directory level from the current file and in a "Data" folder containing the datasets. The reference will be different unless you define the same directory structure.
    trip_df = pd.read_parquet("../data/trips_all.parquet")
    # We want to be able to filter by home location (not given in trip data) and include hh and person details so need hh_all.csv/per_all.csv
    hh_df = pd.read_parquet("../data/hh_all.parquet")
    per_df = pd.read_parquet("../data/per_all.parquet")
    # We'll need household weights for the zone-based model analysis to generate some approximate zonal statistics
    hh_wgt_df = pd.read_parquet("../data/hh_wgt_all.parquet")

    # Merge on UID, which is a combination of a household id and a dataset id 
    trip_df = trip_df.merge(hh_df.loc[:,["UID","hh_cbsa"]], on="UID")

    trip_df.to_parquet("my_hh_trip_data.parquet",index=False)
else:
    trip_df = pd.read_parquet("my_hh_trip_data.parquet")

/tmp/ipykernel_1451/221921571.py:4: DtypeWarning: Columns (26,28,67,91,105,122,124,137,138) have mixed types. Specify dtype option on import or set low_memory=False.
  trip_df = pd.read_csv("/workspaces/CIVE_461/Group Project/Data/trips_all.csv")
/tmp/ipykernel_1451/221921571.py:6: DtypeWarning: Columns (58) have mixed types. Specify dtype option on import or set low_memory=False.
  hh_df = pd.read_csv("/workspaces/CIVE_461/Group Project/Data/hh_all.csv")
/tmp/ipykernel_1451/221921571.py:7: DtypeWarning: Columns (2,3,17,53,61,63,85,86,87,115,143,144,145,158,170) have mixed types. Specify dtype option on import or set low_memory=False.
  per_df = pd.read_csv("/workspaces/CIVE_461/Group Project/Data/per_all.csv")


In [4]:
trip_df.head()

,sampno,perno,tripno,o_locno,locno,strttime,endtime,trvlcmin,distance_mi,trpmiles,...,transit_acct,transit_egrt,transit_waitt,UID,pub_reason,wtperfin5d,wttrdfin5d,wtperfin,wttrdfin,hh_cbsa
0,30000449,1,1,100,102,829,855,26,9.515,9.515,...,8.291029,11.263992,9.576387,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
1,30000449,1,2,102,1000000,905,910,5,0.307,0.307,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
2,30000449,1,3,1000000,102,1000,1005,5,0.307,0.307,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
3,30000449,1,4,102,1000001,1315,1330,15,1.823,1.823,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260
4,30000449,1,5,1000001,102,1625,1640,15,1.815,1.815,...,6.256466,7.472391,9.479657,30000449.1,NaN,NaN,NaN,NaN,NaN,12260


# Important columns
Some important columns to make note of when developing your models are given below. Note: we could obtain an estimate for transit cost from GTFS data. We'll ignore transit cost for the purposes of the assignment though.
- auto_tt = auto travel time in minutes
- auto_cost = auto cost in dollars
- transit_tt = transit travel time in minutes
- transit_acct = transit access time in minutes
- transit_egrt = transit egress time in minutes
- transit_waitt = transit wait time in minutes
- trpmiles = trip distance in miles
- ch_mode = {1 = walk, 2 = bike, 3 = auto, 4 = transit}

Biogeme, like most discrete choice software, requires that all data are numeric. We have several columns with non-numeric entries. We'll have to do some processing to prepare the data for use with Biogeme.

In [5]:
# Filter data for your region
my_trips = trip_df.loc[(trip_df.hh_cbsa=="40900")]

# Home-Based Non-Work Trip Model


In [6]:
hbnw_filt = ((my_trips.trippurp=="HBO") | (my_trips.trippurp=="HBSHOP") | (my_trips.trippurp=="HBSOCREC"))
# This line will filter for non-work trips. It also removes any column with object datatype.
# If a column appears in the data description but not the dataframe after running this line, you may have to
# do some processing to have it available to you for model estimation. E.g., you can change a string trippurp code to
# an integer code - HBO = 1, etc..
hbnw_trips = my_trips.loc[hbnw_filt].select_dtypes(exclude=['object'])
hbnw_trips.fillna(-1, inplace=True)
# Biogeme requires information on if an alternative is available
# Here, I'm assuming that auto/transit modes are always available. We could exclude auto for those without a license.
# I assume that walk/bike are available for trips with distances less than the maximum reported trip distance with that as the chosen mode
hbnw_trips["walk_av"] = (hbnw_trips.trpmiles   <=  hbnw_trips[hbnw_trips.ch_mode==1].trpmiles.max()).astype(int)
hbnw_trips["bike_av"] = (hbnw_trips.trpmiles   <=  hbnw_trips[hbnw_trips.ch_mode==2].trpmiles.max()).astype(int)
hbnw_trips["auto_av"] = 1
hbnw_trips["transit_av"] = 1

We don't want to include a record if it has a negative value in a critical column. We'll filter the dataframe for those important columns.

In [7]:
# Update based on if you add more columns to models that potentially contain negative numbers
check_cols = ["auto_tt","auto_cost", "transit_tt", "transit_acct", "transit_egrt", "transit_waitt", "trpmiles", "ch_mode"]
hbnw_trips = hbnw_trips[hbnw_trips.loc[:,check_cols].sum(axis=1)>-1]

In [8]:
# Need to import data to the Biogeme database structure
database = db.Database("my_hbnw_trips",hbnw_trips)
database.data.head()

# Define Biogeme variables
AUTO_TT = Variable("auto_tt")
AUTO_COST = Variable("auto_cost")
TRANSIT_TT = Variable("transit_tt")
DIST = Variable("trpmiles")
CHOICE = Variable("ch_mode")
WALK_AV = Variable("walk_av")
BIKE_AV = Variable("bike_av")
AUTO_AV = Variable("auto_av")
TRANSIT_AV = Variable("transit_av")

We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [9]:
ASC_WALK = Beta('ASC_WALK',0,None,None,0)
ASC_BIKE = Beta('ASC_BIKE',0,None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',0,None,None,0)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
B_DIST = Beta('B_DIST',0,None,None,0)

The next step is to define the utility function for each alternative mode.

In [10]:
V1 = ASC_WALK + \
     B_DIST * DIST
V2 = ASC_BIKE + \
     B_DIST * DIST
V3 = B_TIME * AUTO_TT + \
     B_COST * AUTO_COST
V4 = ASC_TRANSIT + \
     B_TIME * TRANSIT_TT

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```loglogit``` function to estimate the model.

In [11]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4}

# Associate the availability conditions with the alternatives

av = {1: WALK_AV,
      2: BIKE_AV,
      3: AUTO_AV,
      4: TRANSIT_AV}

logprob_nw = models.loglogit(V,av,CHOICE)
biogeme_nw  = bio.BIOGEME(database,logprob_nw)
biogeme_nw.modelName = "01logit_nw"
results_nw = biogeme_nw.estimate()

The results of the model estimation can then be printed to the terminal.

In [12]:
# Get the results in a pandas table
# Rob. Std. err are standard errors that are robust to heteroskadastic errors.
# This knowledge is beyond the CIVE461 scope.
pandasResults_nw = results_nw.getEstimatedParameters()
print(pandasResults_nw)

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE    -2.026154      0.749845    -2.702097  6.890373e-03
ASC_TRANSIT -4.483413      0.748666    -5.988536  2.117388e-09
ASC_WALK    -0.154999      0.747977    -0.207224  8.358351e-01
B_COST       0.000133      0.002686     0.049644  9.604059e-01
B_DIST      -0.876204      0.086946   -10.077560  0.000000e+00
B_TIME      -0.017947      0.003445    -5.210010  1.888306e-07


In [19]:

asc_bike_estimate = pandasResults_nw.loc['ASC_BIKE', 'Value']



Estimated value for ASC_BIKE: -2.0261542375308217


**NOTE:** The above model gives appropriate results for the sample CBSA! This may not be true for your dataset. DO NOT submit results with positive cost, distance, or time parameters.

Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

We have to redefine the model for it to run in a Jupyter notebook, so there is some repetition in the following code. Note: the parameter definitions have been slightly altered to use the calculated betas. These are stored in a dictionary and the values can be accessed through reference to the appropriate dictionary keys.

In [13]:
# Get the beta parameters estimated in previous step
betas = results_nw.getBetaValues()

ASC_WALK = Beta('ASC_WALK',betas['ASC_WALK'],None,None,0)
ASC_BIKE = Beta('ASC_BIKE',betas['ASC_BIKE'],None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',betas['ASC_TRANSIT'],None,None,0)
B_TIME = Beta('B_TIME',betas['B_TIME'],None,None,0)
B_COST = Beta('B_COST',betas['B_COST'],None,None,0)
B_DIST = Beta('B_DIST',betas['B_DIST'],None,None,0)

prob1_nw = models.logit(V,av,1)
prob2_nw = models.logit(V,av,2)
prob3_nw = models.logit(V,av,3)
prob4_nw = models.logit(V,av,4)

We can then calculate the set of elasticities. We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [23]:
logitelas1_nw = WALK_AV * (1.0 - prob1_nw) * DIST * B_DIST
logitelas2_nw = BIKE_AV * (1.0 - prob2_nw) * DIST * B_DIST
logitelas3_nw = (1.0 - prob3_nw) * AUTO_TT * B_TIME
logitelas4_nw = (1.0 - prob4_nw) * TRANSIT_TT * B_TIME

simulate_nw = {'Prob. walk': prob1_nw,
            'Prob. bike': prob2_nw,
            'Prob. auto':prob3_nw,
            'Prob. transit':prob4_nw,
            'logit elas. 1':logitelas1_nw,
            'logit elas. 2':logitelas2_nw,
            'logit elas. 3':logitelas3_nw,
            'logit elas. 4':logitelas4_nw}

parameter_values = {
    'ASC_BIKE': pandasResults_nw.loc['ASC_BIKE', 'Value'],
    'ASC_TRANSIT': pandasResults_nw.loc['ASC_TRANSIT', 'Value'],
    'ASC_WALK': pandasResults_nw.loc['ASC_WALK', 'Value'],
    'B_COST': pandasResults_nw.loc['B_COST', 'Value'],
    'B_DIST': pandasResults_nw.loc['B_DIST', 'Value'],
    'B_TIME': pandasResults_nw.loc['B_TIME', 'Value']
}

biogeme_nw  = bio.BIOGEME(database,simulate_nw)
biogeme_nw.modelName = "01logit_simul_nw"
sim_results_nw = biogeme_nw.simulate(theBetaValues=parameter_values)
print("Non-work results=",sim_results_nw.describe())

The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 2[1.0]-17[1.0]-18[1.0]-20[1.0]-21[1.0]-28[1.0]-29[1.0]-39[1.0]-43[1.0]-44[1.0]-57[1.0]-62[1.0]-109[1...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 57[2.0]-62[2.0]-144[2.0]-299[2.0]-415[2.0]-417[2.0]-419[2.0]-487[2.0]-488[2.0]-536[2.0]-537[2.0]-593...
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 2[1.0]-17[1.0]-18[1.0]-20[1.0]-21[1.0]-28[1.0]-29[1.0]-39[1.0]-43[1.0]-44[1.0]-57[1.0]-62[1.0]-109[1...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 57[2.0]-62[2.0]-144[2.0]-299[2.0]-415[2.0]-417[2.0]-419[2.0]-487[2.0]-488[2.0]-536[2.0]-537[2.0]-593...
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 2[1.0]-17[1.0]-18[1.0]-20[1.0]-21[1.0]-28[1.0]-29[1.0]-39[1.0]-43[1.0]-44[1.0]-57[1.

Non-work results=          Prob. walk    Prob. bike    Prob. auto  Prob. transit  logit elas. 1  \
count  13070.000000  13070.000000  13070.000000   13070.000000   13070.000000   
mean       0.129227      0.019894      0.844146       0.006733      -2.327954   
std        0.141299      0.021752      0.163244       0.023949       2.533587   
min        0.000000      0.000000      0.000102       0.000002     -10.514151   
25%        0.003064      0.000472      0.723585       0.004743      -3.512803   
50%        0.072182      0.011112      0.909053       0.005560      -1.428134   
75%        0.233885      0.036006      0.988350       0.006151      -0.356491   
max        0.866322      0.133366      0.999091       0.903233       1.054175   

       logit elas. 2  logit elas. 3  logit elas. 4  
count   13070.000000   13070.000000   13070.000000  
mean       -5.043580      -0.039682      -0.813943  
std         8.063592       0.240813       0.335302  
min       -71.848746     -12.381855     

In [25]:
# We calculate the aggregate elasticities

# First the denominator of the aggregate elasticity expression.
denominator_walk_nw = sim_results_nw['Prob. walk'].sum()
denominator_bike_nw = sim_results_nw['Prob. bike'].sum()
denominator_auto_nw = sim_results_nw['Prob. auto'].sum()
denominator_transit_nw = sim_results_nw['Prob. transit'].sum()

# And now the aggregate elasticities themselves.
direct_elas_term_walk_dist_nw = (
    sim_results_nw['Prob. walk']
    * sim_results_nw['logit elas. 1']
    / denominator_walk_nw
).sum()
print(
    f'Aggregate direct point elasticity of walk wrt distance for non-work: '
    f'{direct_elas_term_walk_dist_nw:.3g}'
)

direct_elas_term_bike_dist_nw = (
    sim_results_nw['Prob. bike']
    * sim_results_nw['logit elas. 2']
    / denominator_bike_nw
).sum()
print(
    f'Aggregate direct point elasticity of bike wrt distance for non-work: '
    f'{direct_elas_term_bike_dist_nw:.3g}'
)

direct_elas_term_auto_time_nw = (
    sim_results_nw['Prob. auto']
    * sim_results_nw['logit elas. 3']
    / denominator_auto_nw
).sum()
print(
    f'Aggregate direct point elasticity of auto wrt time for non-work: '
    f'{direct_elas_term_auto_time_nw:.3g}'
)

direct_elas_term_transit_time_nw = (
    sim_results_nw['Prob. transit']
    * sim_results_nw['logit elas. 4']
    / denominator_transit_nw
).sum()
print(
    f'Aggregate direct point elasticity of transit wrt time for non-work: '
    f'{direct_elas_term_transit_time_nw:.3g}'
)

Aggregate direct point elasticity of walk wrt distance for non-work: -0.805
Aggregate direct point elasticity of bike wrt distance for non-work: -0.933
Aggregate direct point elasticity of auto wrt time for non-work: -0.0276
Aggregate direct point elasticity of transit wrt time for non-work: -0.868


# Home-Based Work Trip Model

In [26]:
hbw_filt = (my_trips.trippurp=="HBW")
hbw_trips = my_trips.loc[hbw_filt].select_dtypes(exclude=['object'])
hbw_trips.fillna(-1, inplace=True)
hbw_trips["walk_av"] = (hbw_trips.trpmiles   <=  hbw_trips[hbw_trips.ch_mode==1].trpmiles.max()).astype(int)
hbw_trips["bike_av"] = (hbw_trips.trpmiles   <=  hbw_trips[hbw_trips.ch_mode==2].trpmiles.max()).astype(int)
hbw_trips["auto_av"] = 1
hbw_trips["transit_av"] = 1

We don't want to include a record if it has a negative value in a critical column. We'll filter the dataframe for those important columns.

In [27]:
# Update based on if you add more columns to models that potentially contain negative numbers
check_cols = ["auto_tt","auto_cost", "transit_tt", "transit_acct", "transit_egrt", "transit_waitt", "trpmiles", "ch_mode"]
hbw_trips = hbw_trips[hbw_trips.loc[:,check_cols].sum(axis=1)>-1]

In [28]:
# Need to import data to the Biogeme database structure
database = db.Database("my_hbw_trips",hbw_trips)
database.data.head()

# Define Biogeme variables
AUTO_TT = Variable("auto_tt")
AUTO_COST = Variable("auto_cost")
TRANSIT_TT = Variable("transit_tt")
DIST = Variable("trpmiles")
CHOICE = Variable("ch_mode")
WALK_AV = Variable("walk_av")
BIKE_AV = Variable("bike_av")
AUTO_AV = Variable("auto_av")
TRANSIT_AV = Variable("transit_av")

We can then define the names and starting values for the model parameters. There are 5 parameters defined for the base Swiss Metro model. Biogeme uses a constrained optimization to reduce the search space. The Beta function is called as follows:
```python
Beta(parameter name AS str, initial value AS int, lower bound AS int, upper bound AS int, fixed AS int)
```
If there are no bounds on the parameter search space, the use may enter: ```None```. A value of ```0``` for the ```fixed``` parameter will allow Biogeme to estimate the parameter. In this case, we have only fixed the ASC for the swiss metro mode, to maintain model indentification.

In [29]:
ASC_WALK = Beta('ASC_WALK',0,None,None,0)
ASC_BIKE = Beta('ASC_BIKE',0,None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',0,None,None,0)
B_TIME = Beta('B_TIME',0,None,None,0)
B_COST = Beta('B_COST',0,None,None,0)
B_DIST = Beta('B_DIST',0,None,None,0)

The next step is to define the utility function for each alternative mode.

In [30]:
V1 = ASC_WALK + \
     B_DIST * DIST
V2 = ASC_BIKE + \
     B_DIST * DIST
V3 = B_TIME * AUTO_TT + \
     B_COST * AUTO_COST
V4 = ASC_TRANSIT + \
     B_TIME * TRANSIT_TT

Biogeme has a variety of built-in discrete choice models, which the user can leverage. However, it also allows the user to define their own likelihood function if it is not available in the built-in model library. In this case, we use the built-in ```bioLogLogit``` function to estimate the model.

In [31]:
# Associate utility functions with the numbering of alternatives
V = {1: V1,
     2: V2,
     3: V3,
     4: V4}

# Associate the availability conditions with the alternatives

av = {1: WALK_AV,
      2: BIKE_AV,
      3: AUTO_AV,
      4: TRANSIT_AV}

logprob_w = models.loglogit(V,av,CHOICE)
biogeme_w  = bio.BIOGEME(database,logprob_w)
biogeme_w.modelName = "01logit"
results_w = biogeme_w.estimate()

The results of the model estimation can then be printed to the terminal.

In [32]:
# Get the results in a pandas table
pandasResults_w = results_w.getEstimatedParameters()
print(pandasResults_w)

                Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BIKE    -6.319393      2.123117    -2.976470  2.915873e-03
ASC_TRANSIT -7.882309      2.122634    -3.713456  2.044481e-04
ASC_WALK    -5.931132      2.114021    -2.805616  5.022054e-03
B_COST      -0.017102      0.007676    -2.227955  2.588349e-02
B_DIST      -0.381231      0.069044    -5.521551  3.360199e-08
B_TIME      -0.006599      0.003101    -2.128315  3.331098e-02


**NOTE:** The above model gives appropriate results for the sample CBSA! This may not be true for your dataset. DO NOT submit results with positive cost, distance, or time parameters.

Biogeme also includes functionalities to compute elasticties for model variables. The elasticity can be calculated directly by the user, but Biogeme also includes functions for calculating derivatives for the user.

We have to redefine the model for it to run in a Jupyter notebook, so there is some repetition in the following code. Note: the parameter definitions have been slightly altered to use the calculated betas. These are stored in a dictionary and the values can be accessed through reference to the appropriate dictionary keys.

In [33]:
# Get the beta parameters estimated in previous step
betas = results_w.getBetaValues()

ASC_WALK = Beta('ASC_WALK',betas['ASC_WALK'],None,None,0)
ASC_BIKE = Beta('ASC_BIKE',betas['ASC_BIKE'],None,None,0)
ASC_TRANSIT = Beta('ASC_TRANSIT',betas['ASC_TRANSIT'],None,None,0)
B_TIME = Beta('B_TIME',betas['B_TIME'],None,None,0)
B_COST = Beta('B_COST',betas['B_COST'],None,None,0)
B_DIST = Beta('B_DIST',betas['B_DIST'],None,None,0)

prob1_w = models.logit(V,av,1)
prob2_w = models.logit(V,av,2)
prob3_w = models.logit(V,av,3)
prob4_w = models.logit(V,av,4)

We can then calculate the set of elasticities. We can then complete the process using the .simulate() function. This will calculate the probabilities and elasticities for each observation. The Pandas .describe() function is useful to print the statistics for each of these calculated values (i.e. mean, minimum, maximum, standard deviation, percentiles).

In [35]:
logitelas1_w = WALK_AV * (1.0 - prob1_w) * DIST * B_DIST
logitelas2_w = BIKE_AV * (1.0 - prob2_w) * DIST * B_DIST
logitelas3_w = (1.0 - prob3_w) * AUTO_TT * B_TIME
logitelas4_w = (1.0 - prob4_w) * TRANSIT_TT * B_TIME

simulate_w = {'Prob. walk': prob1_w,
            'Prob. bike': prob2_w,
            'Prob. auto':prob3_w,
            'Prob. transit':prob4_w,
            'logit elas. 1':logitelas1_w,
            'logit elas. 2':logitelas2_w,
            'logit elas. 3':logitelas3_w,
            'logit elas. 4':logitelas4_w}

parameter_values = {
    'ASC_BIKE': pandasResults_nw.loc['ASC_BIKE', 'Value'],
    'ASC_TRANSIT': pandasResults_nw.loc['ASC_TRANSIT', 'Value'],
    'ASC_WALK': pandasResults_nw.loc['ASC_WALK', 'Value'],
    'B_COST': pandasResults_nw.loc['B_COST', 'Value'],
    'B_DIST': pandasResults_nw.loc['B_DIST', 'Value'],
    'B_TIME': pandasResults_nw.loc['B_TIME', 'Value']
}

biogeme_w  = bio.BIOGEME(database,simulate_w)
biogeme_w.modelName = "01logit_simul"
sim_results_w = biogeme_w.simulate(theBetaValues=parameter_values)
print("Results=",sim_results_w.describe())

The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 0[1.0]-1[1.0]-2[1.0]-3[1.0]-4[1.0]-7[1.0]-8[1.0]-9[1.0]-11[1.0]-12[1.0]-15[1.0]-16[1.0]-17[1.0]-18[1...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 2[2.0]-3[2.0]-7[2.0]-8[2.0]-9[2.0]-11[2.0]-12[2.0]-15[2.0]-16[2.0]-31[2.0]-32[2.0]-35[2.0]-42[2.0]-4...
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 0[1.0]-1[1.0]-2[1.0]-3[1.0]-4[1.0]-7[1.0]-8[1.0]-9[1.0]-11[1.0]-12[1.0]-15[1.0]-16[1.0]-17[1.0]-18[1...
The chosen alternative [`2.0`] is not available for the following observations (rownumber[choice]): 2[2.0]-3[2.0]-7[2.0]-8[2.0]-9[2.0]-11[2.0]-12[2.0]-15[2.0]-16[2.0]-31[2.0]-32[2.0]-35[2.0]-42[2.0]-4...
The chosen alternative [`1.0`] is not available for the following observations (rownumber[choice]): 0[1.0]-1[1.0]-2[1.0]-3[1.0]-4[1.0]-7[1.0]-8[1.0]-9[1.0]-11[1.0]-12[1.0]-15[1.0]-16[1

Results=         Prob. walk   Prob. bike   Prob. auto  Prob. transit  logit elas. 1  \
count  2830.000000  2830.000000  2830.000000    2830.000000    2830.000000   
mean      0.030522     0.007179     0.955582       0.006717      -0.079324   
std       0.091828     0.014476     0.106578       0.020159       0.264125   
min       0.000000     0.000000     0.027828       0.000530      -1.395430   
25%       0.000000     0.000000     0.986866       0.004888       0.000000   
50%       0.000000     0.000072     0.993264       0.005684       0.000000   
75%       0.000000     0.005736     0.994843       0.006589       0.000000   
max       0.660334     0.101656     0.999470       0.972172       0.000000   

       logit elas. 2  logit elas. 3  logit elas. 4  
count    2830.000000    2830.000000    2830.000000  
mean       -3.236109      -0.015568      -1.051970  
std         3.519129       0.186209       0.345950  
min       -11.661378      -9.421459      -3.766778  
25%        -5.848347   

In [36]:
# We calculate the aggregate elasticities

# First the denominator of the aggregate elasticity expression.
denominator_walk_w = sim_results_w['Prob. walk'].sum()
denominator_bike_w = sim_results_w['Prob. bike'].sum()
denominator_auto_w = sim_results_w['Prob. auto'].sum()
denominator_transit_w = sim_results_w['Prob. transit'].sum()

# And now the aggregate elasticities themselves.
direct_elas_term_walk_dist_w = (
    sim_results_w['Prob. walk']
    * sim_results_w['logit elas. 1']
    / denominator_walk_w
).sum()
print(
    f'Aggregate direct point elasticity of walk wrt distance: '
    f'{direct_elas_term_walk_dist_w:.3g}'
)

direct_elas_term_bike_dist_w = (
    sim_results_w['Prob. bike']
    * sim_results_w['logit elas. 2']
    / denominator_bike_w
).sum()
print(
    f'Aggregate direct point elasticity of bike wrt distance: '
    f'{direct_elas_term_bike_dist_w:.3g}'
)

direct_elas_term_auto_time_w = (
    sim_results_w['Prob. auto']
    * sim_results_w['logit elas. 3']
    / denominator_auto_w
).sum()
print(
    f'Aggregate direct point elasticity of auto wrt time: '
    f'{direct_elas_term_auto_time_w:.3g}'
)

direct_elas_term_transit_time_w = (
    sim_results_w['Prob. transit']
    * sim_results_w['logit elas. 4']
    / denominator_transit_w
).sum()
print(
    f'Aggregate direct point elasticity of transit wrt time: '
    f'{direct_elas_term_transit_time_w:.3g}'
)

Aggregate direct point elasticity of walk wrt distance: -0.575
Aggregate direct point elasticity of bike wrt distance: -1.35
Aggregate direct point elasticity of auto wrt time: -0.0097
Aggregate direct point elasticity of transit wrt time: -1.04


# Supplemental notes
Biogeme provides many other functions for viewing results: <br> getGeneralStatistics() <br> getEstimatedParameters() <br> getCorrelationResults() <br> getBetaValues() <br> getRobustVarCovar() <br> getBetasForSensitivityAnalysis()

The manuals are also quite helpful: <br>
http://biogeme.epfl.ch/documents.html